### Compilations

In [ ]:
!apt update && apt install -y libsm6 libxext6
!apt-get install -y libxrender-dev

In [ ]:
%cd /workdir/msc/RotationDetection/libs/utils/cython_utils
!rm *.so
!rm *.c
!rm *.cpp
!python setup.py build_ext --inplace

In [ ]:
%cd /workdir/msc/RotationDetection/libs/utils/
!rm *.so
!rm *.c
!rm *.cpp
!python setup.py build_ext --inplace

### Dataset

In [ ]:
# change train and test paths inside the script
%cd /workdir/msc/RotationDetection/dataloader/dataset/UFRGS_CELL
!python convert_ann2dota.py

In [ ]:
# change root path inside the script
%cd /workdir/msc/RotationDetection/dataloader/dataset/UFRGS_CELL
!python convert2jpg.py

In [ ]:
# change train path inside the script
%cd /workdir/msc/RotationDetection/dataloader/dataset/UFRGS_CELL
!python data_crop.py

In [ ]:
# change test path inside the script
%cd /workdir/msc/RotationDetection/dataloader/dataset/UFRGS_CELL
!python txt2xml.py

In [ ]:
%cd /workdir/msc/RotationDetection/dataloader/dataset/
!python convert_data_to_tfrecord.py --VOC_dir='/workdir/datasets/msc/UFRGS_CELL_2classes/crop' \
                                    --xml_dir='labeltxt' \
                                    --image_dir='images' \
                                    --save_name='train'  \
                                    --img_format='.png'  \
                                    --save_dir='/workdir/datasets/msc/' \
                                    --dataset='UFRGS_CELL' \

## Train

In [20]:
# change the path to the detector!
%cd /workdir/msc/RotationDetection/tools/rsdet
!python train_5p.py

************************************************************************
2021-09-08 18:02:51: global_step:110940  current_step:110940
speed: 0.149s, remaining training time: 00:00:47:27
total_losses:0.253
cls_loss:0.107
reg_loss:0.145

************************************************************************
2021-09-08 18:02:54: global_step:110960  current_step:110960
speed: 0.148s, remaining training time: 00:00:47:03
total_losses:0.028
cls_loss:0.015
reg_loss:0.013

************************************************************************
2021-09-08 18:02:58: global_step:110980  current_step:110980
speed: 0.148s, remaining training time: 00:00:46:58
total_losses:0.110
cls_loss:0.023
reg_loss:0.086

************************************************************************
2021-09-08 18:03:04: global_step:111020  current_step:111020
speed: 0.145s, remaining training time: 00:00:45:49
total_losses:0.034
cls_loss:0.018
reg_loss:0.017

********************************************************

## Test

| Model      | AP50 normal | AP50 mitoses | P50 normal | R50 normal | P50 mitoses | R50 mitoses |   AP50    |   AP75    |   AP50:95   |
|----------- | ----------- | ------------ | ---------- | ---------- | ----------- | ----------- |   -----   |   -----   |   -------   |
| RetinaNet  |   75.25     |   56.47      | 37.64      | 85.00      | 10.99       | 82.85       |   65.86   |   19.17   |   29.56     |
| R3Det      |   74.97     |   63.21      | 32.68      | 83.42      | 14.35       | 84.76       |   69.09   |   17.91   |   30.89     |
| R3Det DCL  |   75.50     |   62.63      | 31.58      | 83.54      | 15.75       | 84.76       |   69.06   | **20.57** |   30.73     |
| DCL        |   71.81     |   63.35      | 39.58      | 81.62      | 16.50       | 80.95       |   67.58   |   17.74   |   28.48     |
| CSL        |   72.42     |   53.98      | 35.20      | 84.10      | 10.94       | 84.76       |   63.20   |   18.59   |   27.00     |
| R2CNN      | **77.33**   | **69.65**    | **67.49**  | 82.41      | **52.79**   | 80.95       | **73.49** |   17.30   |   29.71     |
| Refine RetinaNet | 75.15 |   66.75      | 27.93      | **85.68**  | 11.94       | **89.52**   |   70.95   |   18.86   | **31.00**   |
| RSDet      |   74.33     |   63.60      | 38.65      | 83.31      | 9.63        | 87.62       |   68.97   |   16.15   |   29.86     |

In [8]:
%cd /workdir/msc/RotationDetection/tools/retinanet
!python test_ufrgscell.py --img_dir='/workdir/datasets/msc/UFRGS_CELL_2classes/test/imgs'  \
                          --gpu=0 \
                          --image_ext='.jpg' \
                          --test_annotation_path='/workdir/datasets/msc/UFRGS_CELL_2classes/test/xml_rotdet' \
                          none

/workdir/msc/RotationDetection/tools/retinanet
/workdir/msc/RotationDetection
/workdir/msc/RotationDetection/output/trained_weights/RetinaNet_UFRGS_CELL_smooth_l1_loss


******************************
 CHECKPOINT PATH: /workdir/msc/RotationDetection/output/trained_weights/RetinaNet_UFRGS_CELL_smooth_l1_loss/UFRGS_CELL_199999model.ckpt

******************************
model restore from : /workdir/msc/RotationDetection/output/trained_weights/RetinaNet_UFRGS_CELL_smooth_l1_loss/UFRGS_CELL_199999model.ckpt

**********
Restored model
/workdir/msc/RotationDetection/output/trained_weights/RetinaNet_UFRGS_CELL_smooth_l1_loss/UFRGS_CELL_199999model.ckpt
**********
Eval image exp_M3_Mauricio_mcf7_frame_17: 100%|█| 30/30 [00:12<00:00,  2.49it/s]
********************
rotation eval:
Threshold:  0.5
cls : normal_cell|| Recall: 0.850056368827445 || Precison: 0.3764353469795307|| AP: 0.7525355201434214
mAP is : 0.7525355201434214


Threshold:  0.5
cls : mitoses|| Recall: 0.8285714206802722 || Precison

## Inference

In [ ]:
import os
import cv2
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

path_labels = '/workdir/datasets/msc/UFRGS_CELL_2classes/test/annotations/dota_format'
path_draws = '/workdir/msc/RotationDetection/tools/gwd/test_ufrgscell/RetinaNet_UFRGS_CELL_smooth_l1_loss/ufrgscell_img_vis'

path_labels = sorted(glob(os.path.join(path_labels, '*.txt')))
path_draws  = sorted(glob(os.path.join(path_draws, '*.jpg')))

for i in range(len(path_labels)):
    path_ann = path_labels[i]
    path_img = path_draws[i]

    img = cv2.imread(path_img)[:,:,::-1]
    h,w,c = img.shape

    labels = []
    for lb in open(path_ann, 'r').read().split('\n')[2:-1]:
        lb = np.reshape(list(map(float, lb.split(' ')[:-2])), (-1,2))
        labels.append(lb)
    labels = np.int0(labels)

    draw = np.copy(img)
    cv2.drawContours(draw, labels, -1, (255,0,0), 2)

    plt.figure(figsize=(15,15))
    plt.imshow(draw)

In [ ]:
%cd /workdir/msc/RotationDetection
!python run_inference.py \
--img_dir='/workdir/datasets/msc/UFRGS_CELL_2classes/test/imgs'  \
--gpu=0 \
--image_ext='.jpg' \
--test_annotation_path='/workdir/datasets/msc/UFRGS_CELL_2classes/test/xml_rotdet' \
csl